In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer

In [4]:


def ising_hamiltonian(num_qubits):
    coeffs = [1.0] * (num_qubits - 1)
    obs_list = [qml.PauliZ(i) @ qml.PauliZ(i + 1) for i in range(num_qubits - 1)]
    return qml.Hamiltonian(coeffs, obs_list)

# Define the number of qubits
num_qubits = 5

# Define the quantum device
dev = qml.device("qiskit.aer", wires=num_qubits)

# Define the ansatz circuit
def ansatz(params, wires):
    for i in range(0, len(wires), 2):
        qml.RY(params[i], wires=wires[i])
        if i + 1 < len(wires):
            qml.RY(params[i + 1], wires=wires[i + 1])
            qml.CNOT(wires=[wires[i], wires[i + 1]])

# Define the quantum node
@qml.qnode(dev)
def circuit(params):
    ansatz(params, wires=range(num_qubits))
    return qml.expval(ising_hamiltonian(num_qubits))

# Define the cost function
def cost(params):
    return circuit(params)

# Set up the optimizer
opt = AdamOptimizer(0.1)

# Initialize random parameters
params = np.random.rand(2 * num_qubits - 1)

# Optimize the parameters to find the ground state
iterations = 10000
for i in range(iterations):
    params = opt.step(cost, params)
    if i % 10 == 0:
        energy = circuit(params)
        print(f"Iteration {i+1}/{iterations} - Current energy: {energy}")

# Print the final result
final_energy = circuit(params)
print(f"Optimized parameters: {params}")
print(f"Ground state energy: {final_energy}")

# Find states at different energies
energies_to_check = [-2.0, -1.5, -1.0, -0.5]
for target_energy in energies_to_check:
    params = opt.step(lambda p: (circuit(p) - target_energy)**2, params)
    state = dev._state
    print(f"Target energy: {target_energy}")
    print(f"State at target energy: {state}")

Iteration 1/10000 - Current energy: 1.529296875
Iteration 11/10000 - Current energy: -0.943359375
Iteration 21/10000 - Current energy: -2.560546875
Iteration 31/10000 - Current energy: -3.67578125
Iteration 41/10000 - Current energy: -3.91015625
Iteration 51/10000 - Current energy: -3.923828125
Iteration 61/10000 - Current energy: -3.998046875
Iteration 71/10000 - Current energy: -3.984375
Iteration 81/10000 - Current energy: -3.99609375
Iteration 91/10000 - Current energy: -4.0
Iteration 101/10000 - Current energy: -4.0
Iteration 111/10000 - Current energy: -3.998046875
Iteration 121/10000 - Current energy: -4.0
Iteration 131/10000 - Current energy: -3.998046875
Iteration 141/10000 - Current energy: -4.0
Iteration 151/10000 - Current energy: -4.0
Iteration 161/10000 - Current energy: -4.0
Iteration 171/10000 - Current energy: -3.998046875
Iteration 181/10000 - Current energy: -4.0
Iteration 191/10000 - Current energy: -4.0
Iteration 201/10000 - Current energy: -4.0
Iteration 211/10000

KeyboardInterrupt: 